# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [4]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [5]:
document_tree = ET.parse( 'data/mondial_database_less.xml' )

In [13]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text
    #print child.find('infant_mortality').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [7]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
from xml.etree import ElementTree as ET
import pandas as pd
document = ET.parse( './data/mondial_database.xml' )
document_root = document.getroot()

In [19]:
# 10 Countries with the lowest infant mortality rate 
df_country_imr = pd.DataFrame(columns=["country", "infantMortality"])

for country in document.getiterator("country"):
    country_name = country.find("name").text
    if country.find("infant_mortality") is not None:
        imr = country.find("infant_mortality").text
        df_country_imr = df_country_imr.append(
            {
                "country": country_name,
                "infantMortality": imr
            }, ignore_index=True)
        
df_country_imr["infantMortality"] = pd.to_numeric(df_country_imr["infantMortality"])
print 'Top 10 countries with lowest infant mortality rate'
df_country_imr.sort_values('infantMortality').head(10)

Top 10 countries with lowest infant mortality rate


,country,infantMortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [10]:
# Top 10 cities with the largest population
# dataframe for the population measurement with year and country 
df_city_pop = pd.DataFrame(columns=["city", "year", "population"])
for country in document_root.getiterator("country"):
    for city in country.iter("city"):
        city_name = city.find("name").text
# if population is not none and given in the file it reads the population        
        if city.findall("population") is not None:
            for pop in city.findall("population"):
                year = pop.attrib["year"]
                population = pop.text
# Append the information to the data frame            
                df_city_pop = df_city_pop.append(
                {
                    "city": city_name,
                    "year": year,
                    "population": population
                }, ignore_index=True)
        
        
            
# check the data by displaying it
df_city_pop.head()



,city,year,population
0,Tirana,1987,192000
1,Tirana,1990,244153
2,Tirana,2011,418495
3,Shkodër,1987,62000
4,Shkodër,2011,77075


In [18]:
# make a copy of dataframe to save it for changes
df_city_pop_c = df_city_pop.copy()

# dataframe for the latest population measurement with year and city 
df_latest_city_pop = pd.DataFrame(columns=["city", "year", "population"])

# change population column to numeric
df_city_pop_c["population"] = pd.to_numeric(df_city_pop_c["population"])

# select all unique cities
cities = df_city_pop["city"].unique()

# assamble all cities with latest population measurement in one dataframe
for city in cities:
    city_pop = df_city_pop_c[df_city_pop_c.city == city]
    latest_measurement = city_pop[city_pop["year"] == city_pop["year"].max()]
    df_latest_city_pop = df_latest_city_pop.append(latest_measurement)

# sort values by population in descending order and take the first 10 elements
print '10 cities with largest population'
df_latest_city_pop.sort_values("population", ascending=False)[0:10]


10 cities with largest population


,city,year,population
3750,Shanghai,2010,22315474.0
2607,Istanbul,2012,13710512.0
4303,Mumbai,2011,12442373.0
1546,Moskva,2013,11979529.0
3746,Beijing,2010,11716620.0
8208,São Paulo,2010,11152344.0
3754,Tianjin,2010,11090314.0
3364,Guangzhou,2010,11071424.0
4399,Delhi,2011,11034555.0
3371,Shenzhen,2010,10358381.0


In [54]:
## Finding the top 10 ethnic groups
# to solve this, first we need to get the latest population for each country
# After that extract the ethich group names and percentage from each country and calculate the actual values
df_country_pop = pd.DataFrame(columns=["country", "year", "population"])
df_ethnic_group = pd.DataFrame(columns=["Ethnic","population"])
for country in document_root.getiterator("country"):
    country_name = country.find("name").text
    
# if population is not none and given in the file it reads the population        
    if country.findall("population") is not None:
        for pop in country.findall("population"):
            year = pop.attrib["year"]
            population = pop.text
# Append the information to the data frame            
            df_country_pop = df_country_pop.append(
            {
                "country": country_name,
                "year": year,
                "population": population
            }, ignore_index=True)
    if country.findall("ethnicgroup") is not None:
        for perc in country.findall("ethnicgroup"):
            percentage = pd.to_numeric(perc.attrib["percentage"])
            ethnicgroup = perc.text
            ethnicpopulation = ((percentage*100.0)/10000)*(pd.to_numeric(df_country_pop['population'].iloc[-1]))
            df_ethnic_group = df_ethnic_group.append({"Ethnic": ethnicgroup, "population":ethnicpopulation}, ignore_index=True)
            

print 'Top 10 Ethnic Groups'
df_ethnic_group.groupby(['Ethnic']).sum().sort_values(['population'], ascending = False).head(10)


    


Top 10 Ethnic Groups


,population
Ethnic,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [16]:
## Name of the longest river, largest lake and the airport at highest evelation
# Working to get the longest river
# Declaring a new data frame
df_river = pd.DataFrame(columns = ['river', 'country','length'])

# running a loop to get the source country of river and its length
for river in document_root.getiterator('river'):
    riverName = river.find('name').text
    country = river.find('source').get('country')
    if river.find('length') is not None:
        length = river.find('length').text
    else:
        length = '0'
    df_river = df_river.append({'river': riverName, 'country':country, 'length':length},ignore_index = True)

# converting the column into numeric so it can be sorted    
df_river['length'] = pd.to_numeric(df_river['length'])
print 'The longest River'
df_river.sort_values('length', ascending = False).head(1)
    

The longest River


,river,country,length
174,Amazonas,PE,6448.0


In [17]:
## Working to get the largest Lake
# defining a data frame for the lake
df_lake =  pd.DataFrame(columns = ['lake', 'country', 'area'])

# running a loop to get the location of lake and it's area
for lake in document_root.getiterator('lake'):
    lakeName = lake.find('name').text
    country = lake.get('country')
# if area is not present entering value as 0
    if lake.find('area') is not None:
        area = lake.find('area').text
    else:
        area = '0'
    

    df_lake = df_lake.append({'lake': lakeName, 'country':country, 'area':area},ignore_index = True)

# converting the column into numeric so it can be sorted   
df_lake['area'] = pd.to_numeric(df_lake['area'])
print 'Lake with largest Area'
df_lake.sort_values('area', ascending = False).head(1)
    

Lake with largest Area


,lake,country,area
54,Caspian Sea,R AZ KAZ IR TM,386400.0


In [15]:
## Working to get the airport with highest elevation
# defining a data frame for the airport list
df_airport =  pd.DataFrame(columns = ['airport', 'country','city', 'elevation'])
for airport in document_root.getiterator('airport'):
    airportName = airport.find('name').text
    country = airport.get('country')
    city = airport.get('city')
# if eleveation is not present then entering value as -10
    if airport.find('elevation') is not None:
        elevation = airport.find('elevation').text
    else:
        elevation = '-10'
    df_airport = df_airport.append({'airport': airportName, 'country':country,'city':city, 'elevation':elevation},ignore_index = True)

# converting the column into numeric so it can be sorted    
df_airport['elevation'] = pd.to_numeric(df_airport['elevation'])
print 'Airport with the highest elevation'
df_airport.sort_values('elevation', ascending = False).head(1)



Airport with the highest elevation


,airport,country,city,elevation
80,El Alto Intl,BOL,cty-BOL-1,4063.0
